This notebook load the the Mean MEG Data and uses a Random Forest to classify between MCI and Control patients. It is a simple bechmark to measure the performance of other strategies. I have selected the random forest as the benchmarking model because it's simple to set up and doesn't have many hyperparameters to optimize (mtry being the most important one).

Also, it will be useful for getting comfortable with the MLR package's syntax, since I've since now always used caret.

In [1]:
library(tidyverse)
library(mlr)
library(here)
here()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: ParamHelpers
Warning message:
“replacing previous import ‘BBmisc::isFALSE’ by ‘backports::isFALSE’ when loading ‘ParamHelpers’”here() starts at /home/rstudio/alzheimer_meg


[1] "/home/rstudio/alzheimer_meg"

In [ ]:
here()

In [ ]:
mean_data <- readRDS("/home/rstudio/data/mean.rds")
mean_data$class <- factor(mean_data$class)

In [ ]:
task = makeClassifTask(id = "rf_mean_meg_pca", data = mean_data, target = "class")

Here, I create a randomForest learner and set the resampling strategy as a Leave-One-Out Crossvalidation. 

In [ ]:
lrn = makeLearner("classif.xgboost")
rdesc = makeResampleDesc(method = "LOO")

Train the model

In [ ]:
r = resample(learner = lrn, task = task, resampling = rdesc)

The mean MMCE is 0.3560606. Now, we will look at the Confunsion Matrix (class 2 is MCI):

In [ ]:
print("Confusion matrix:")
confusion_matrix <- calculateConfusionMatrix(r$pred)
print(confusion_matrix)
print("Accuracy:")
sum(r$pred$data$truth == r$pred$data$response)/length(r$pred$data$truth)
t(confusion_matrix$result)

In [ ]:
TP <- confusion_matrix$result[2,2]
FP <- confusion_matrix$result[1,2]
TN <- confusion_matrix$result[1,1]
FN <- confusion_matrix$result[2,1]

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 <- 2 * precision * recall / (precision + recall)
f1
precision
recall